# [Based on this guide](https://barrowclift.me/post/wireguard-server-on-macos)

# How RMCS makes it work
- Servers running macOS will not have a .app file. 
- Must run from python source (for using `sudo python rmcc_server.py`) 
- ~~Will have a manual toggle for the Wireguard post-up and post-down commands, along with onscreen walkthrough of when and how to enable/disable from the Wireguard GUI~~
- Have user install `homebrew`, then `python` and `wireguard` via `homebrew` (simple command line), and then we take care of creating all the requirements and setting permissions etc

- Requirements
    - `brew install python` (automatically runs python 3)
    - `brew install wireguard-tools`
    - `sudo python rmcc_server.py`
    
# Postup
```shell
#!/bin/sh

# 1) Preparing the directory which we'll persist the pf token
#    generated by (2) in. That token can then be used by our
#    postdown.sh script to remove the routing rule in the
#    event Wireguard is shut down on the server.
mkdir -p /usr/local/var/run/wireguard
chmod 700 /usr/local/var/run/wireguard

# 2) Dynamically add the NAT rule, enable the firewall, increase
#    its reference count (-E) and persist the reference token
#    generated by the command into pf_wireguard_token.txt,
#    which postdown.sh will reference when Wireguard is shut
#    down.
echo 'nat on en0 from 10.0.10.0/24 to any -> (en0)' | \
    pfctl -a com.apple/wireguard -Ef - 2>&1 | \
    grep 'Token' | \
    sed 's%Token : \(.*\)%\1%' > /usr/local/var/run/wireguard/pf_wireguard_token.txt   
```
    
# Postdown
```shell
#!/bin/sh

# 1) Fetch the pf reference token that was generated on
#    Wireguard startup with postup.sh
TOKEN=`cat /usr/local/var/run/wireguard/pf_wireguard_token.txt`

# 2) Remove the reference (and by extension, the pf rule that
#    generated it). Adding and removing rules by references
#    like this will automatically disable the packet filter
#    firewall if there are no other references left, but will
#    leave it up and intact if there are.
pfctl -X ${TOKEN} || exit 1
rm -f /usr/local/var/run/wireguard/pf_wireguard_token.txt
```

In [ ]:
from pathlib import Path

sh_path = Path("/usr/local/etc/wireguard/")
if not sh_path.exists():
    sh_path.mkdir(parents=True)

# must be run with sudo
# 2) Dynamically add the NAT rule, enable the firewall, increase
#    its reference count (-E) and persist the reference token
#    generated by the command into pf_wireguard_token.txt,
#    which postdown.sh will reference when Wireguard is shut
#    down.
# -a : anchor the app with signature com.apple/wireguard
# -Ef : enable packet filter firewall
# 2>&1 : redirect stderr to stdout (print any errors)

subnet = "10.0.10.0/24"

# all backslashes escaped as \\ so they don't disappear (and wreak havoc)
postup = (f"""#!/bin/sh

# 1) Preparing the directory which we'll persist the pf token
#    generated by (2) in. That token can then be used by our
#    postdown.sh script to remove the routing rule in the
#    event Wireguard is shut down on the server.
mkdir -p /usr/local/var/run/wireguard
chmod 700 /usr/local/var/run/wireguard

# 2) Dynamically add the NAT rule, enable the firewall, increase
#    its reference count (-E) and persist the reference token
#    generated by the command into pf_wireguard_token.txt,
#    which postdown.sh will reference when Wireguard is shut
#    down.
echo 'nat on en0 from {subnet} to any -> (en0)' | \\
    pfctl -a com.apple/wireguard -Ef - 2>&1 | \\
    grep 'Token' | \\
    sed 's%Token : \\(.*\\)%\\1%' > /usr/local/var/run/wireguard/pf_wireguard_token.txt""")

print(postup)

with open('/usr/local/etc/wireguard/postup.sh', 'w') as postup_file:
    postup_file.write(postup)
    
postdown = ("""#!/bin/sh

# 1) Fetch the pf reference token that was generated on
#    Wireguard startup with postup.sh
TOKEN=`cat /usr/local/var/run/wireguard/pf_wireguard_token.txt`

# 2) Remove the reference (and by extension, the pf rule that
#    generated it). Adding and removing rules by references
#    like this will automatically disable the packet filter
#    firewall if there are no other references left, but will
#    leave it up and intact if there are.
pfctl -X ${TOKEN} || exit 1
rm -f /usr/local/var/run/wireguard/pf_wireguard_token.txt""")

print(postdown)

with open('/usr/local/etc/wireguard/postdown.sh', 'w') as postdown_file:
    postdown_file.write(postdown)

# Make executable
```shell
chmod u+x /usr/local/etc/wireguard/*.sh
```

In [ ]:
import os

# make executable
try:
    assert(os.system("chmod u+x /usr/local/etc/wireguard/*.sh") == 0)
except AssertionError:
    print("Could not make file executable. Running app as sudo?")

# Install Wireguard tools
```shell
brew install wireguard-tools
```

User must:
- Install homebrew: https://brew.sh
- Either:
    - Install cakebrew: https://www.cakebrew.com and then install wireguard-tools via the cakebrew
    - Or just `brew install wireguardtools`

# Configure wireguard
```shell
umask 077 # Ensure credentials don't leak from possible race condition.
wg genkey | tee privatekey | wg pubkey > publickey
```

In [ ]:
os.system("umask 077") # Ensure credentials don't leak from possible race condition.
os.system("wg genkey | tee privatekey | wg pubkey > publickey")

with open('privatekey','r') as pk_file:
    s_pk = pk_file.read().strip()
    
with open('publickey','r') as Pk_file:
    s_Pk = Pk_file.read().strip()

Path("privatekey").unlink() # (delete) 
Path("publickey").unlink() # (delete) 

s_pk, s_Pk

In [ ]:
os.system("umask 077") # Ensure credentials don't leak from possible race condition.
os.system("wg genkey | tee privatekey | wg pubkey > publickey")

with open('privatekey','r') as pk_file:
    c_pk = pk_file.read().strip()
    
with open('publickey','r') as Pk_file:
    c_Pk = Pk_file.read().strip()

Path("privatekey").unlink() # (delete) 
Path("publickey").unlink() # (delete) 

c_pk, c_Pk

# Create Server configurations
```python
[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = 10.0.10.0/24
# Substitute with your *server's* private key
PrivateKey = XXX
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = 51820
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 1.0.0.1, 2606:4700:4700::1111
# This ensures our peers continue to report their Wireguard-
# assigned IPs while connected to the VPN. This is required for
# their traffic to get routed correctly by the firewall rules we
# crafted earlier with pf.
PostUp = /usr/sbin/sysctl -w net.inet.ip.forwarding=1
PostUp = /usr/sbin/sysctl -w net.inet6.ip6.forwarding=1
# Adds the firewall routing rule on Wireguard server startup
PostUp = /usr/local/etc/wireguard/postup.sh
# Removes the firewall routing rule on Wireguard server shutdown
PostDown = /usr/local/etc/wireguard/postdown.sh

[Peer]
# Substitute with *this peer*'s public key
PublicKey = XXX
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = 10.0.10.10/32

# Follow the same steps as the [Peer] template above for each
# additional peer you wish to connect to the VPN with.
```

In [ ]:
client_ip = "XXX.XXX.XXX.XXX"
server_ip = "XXX.XXX.XXX.XXX"
port = 55555

In [ ]:
server_config = (f"""
[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = {subnet}
# Substitute with your *server's* private key
PrivateKey = {s_pk}
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = {port}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 8.8.8.8
# This ensures our peers continue to report their Wireguard-
# assigned IPs while connected to the VPN. This is required for
# their traffic to get routed correctly by the firewall rules we
# crafted earlier with pf.
PostUp = /usr/sbin/sysctl -w net.inet.ip.forwarding=1
PostUp = /usr/sbin/sysctl -w net.inet6.ip6.forwarding=1
# Adds the firewall routing rule on Wireguard server startup
PostUp = /usr/local/etc/wireguard/postup.sh
# Removes the firewall routing rule on Wireguard server shutdown
PostDown = /usr/local/etc/wireguard/postdown.sh

[Peer]
# Substitute with *this peer*'s public key
PublicKey = {c_Pk}
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = {client_ip}/32

# Follow the same steps as the [Peer] template above for each
# additional peer you wish to connect to the VPN with.
""")

print(server_config)

with open("/usr/local/etc/wireguard/server.conf", 'w') as conf:
    conf.write(server_config)

# Create Client configurations
```python
[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = 10.0.10.10/32
# Substitute with *this peer's* private key.
PrivateKey = XXX
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 1.0.0.1, 2606:4700:4700::1111

[Peer]
# Substitute with your *server's* public key
PublicKey = XXX
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = XXX.XXX.XXX.XXX:51820
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
```

In [ ]:
# Create Client configurations
client_config = (f"""
[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = {client_ip}/32
# Substitute with *this peer's* private key.
PrivateKey = {c_pk}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 8.8.8.8

[Peer]
# Substitute with your *server's* public key
PublicKey = {s_Pk}
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = {server_ip}:{port}
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
""")

print(client_config)

# Up/down

- Launch with `sudo wg-quick up /usr/local/etc/wireguard/server.conf`
- Halt with `sudo wg-quick down /usr/local/etc/wireguard/server.conf`

In [98]:
up = subprocess.check_output("sudo wg-quick up /usr/local/etc/wireguard/server.conf", shell=True, stderr=subprocess.STDOUT)
print(str(up,'utf-8'))

[#] wireguard-go utun
[+] Interface for server is utun9
[#] wg setconf utun9 /dev/fd/63
[#] ifconfig utun9 inet 10.0.10.0/24 10.0.10.0 alias
[#] ifconfig utun9 up
[#] route -q -n add -inet 10.0.10.10/32 -interface utun9
[#] networksetup -getdnsservers USB 10/100/1000 LAN
[#] networksetup -getsearchdomains USB 10/100/1000 LAN
[#] networksetup -getdnsservers Wi-Fi
[#] networksetup -getsearchdomains Wi-Fi
[#] networksetup -getdnsservers Bluetooth PAN
[#] networksetup -getsearchdomains Bluetooth PAN
[#] networksetup -getdnsservers Thunderbolt Bridge
[#] networksetup -getsearchdomains Thunderbolt Bridge
[#] networksetup -getdnsservers Philharmonic External
[#] networksetup -getsearchdomains Philharmonic External
[#] networksetup -getdnsservers Philharmonic Local
[#] networksetup -getsearchdomains Philharmonic Local
[#] networksetup -setdnsservers Bluetooth PAN 1.1.1.1 8.8.8.8
[#] networksetup -setsearchdomains Bluetooth PAN Empty
[#] networksetup -setdnsservers Philharmonic Local 1.1.1.1 8.

In [101]:
show = subprocess.check_output("sudo wg show", shell=True, stderr=subprocess.STDOUT)
print(str(output,'utf-8')) # returns empty if 

In [100]:
down = subprocess.check_output("sudo wg-quick down /usr/local/etc/wireguard/server.conf", shell=True, stderr=subprocess.STDOUT)
print(str(down,'utf-8'))

[+] Interface for server is utun9
[#] rm -f /var/run/wireguard/utun9.sock
[#] rm -f /var/run/wireguard/server.name
[#] /usr/local/etc/wireguard/postdown.sh
No ALTQ support in kernel
ALTQ related functions disabled
disable request successful. 11 more pf enable reference(s) remaining, pf still enabled.



In [104]:
def thing():
    # Create Client configurations
    client_config = (f"""
[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = {client_ip}/32
# Substitute with *this peer's* private key.
PrivateKey = {c_pk}
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 8.8.8.8

[Peer]
# Substitute with your *server's* public key
PublicKey = {s_Pk}
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = {server_ip}:{port}
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
    """)

    print(client_config)

In [105]:
thing()


[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = 10.0.10.10/32
# Substitute with *this peer's* private key.
PrivateKey = SJEw2D5rM6UDJDcq0rJ7O9nu4h6GB1m+mrDbDqDw9Gs=
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 8.8.8.8

[Peer]
# Substitute with your *server's* public key
PublicKey = NSgSVhvSLUgAWvjJEzQzacrcfqoH24CHd3qnL9q+LRI=
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = 73.162.86.31:55555
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
    
